In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import cvxpy as cp


In [12]:
df = pd.read_csv("data/final_data.csv")
data_df = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
data_matrix = data_df.values
print(data_df.shape)

(7162, 1590)


In [13]:
delta=df[['u_id', 'a_id', 'score']]
delta

u_id   a_id  score
0          0  12403      6
1        104  12403      9
2        109  12403      8
3        460  12403      7
4       1336  12403     10
...      ...    ...    ...
59149  15121   3230      8
59150  15698   3230      8
59151  22915   3230      7
59152  24855   3230      6
59153  35762   3230      7

[59154 rows x 3 columns]

In [14]:
user_id_index_map = {u_id: idx for idx, u_id in enumerate(data_df.index)}
anime_id_index_map = {a_id: idx for idx, a_id in enumerate(data_df.columns)}

def user_id_to_index(uid):
    return user_id_index_map.get(uid, -1)  # returns -1 if uid not found

def anime_id_to_index(aid):
    return anime_id_index_map.get(aid, -1) # returns -1 if aid not found

In [15]:
anime_id_to_index(35760)

1413

In [16]:
#This takes in dataframe input
def nuclear_norm_model_df(df):
    user_list = sorted(df['u_id'].unique())
    anime_list = sorted(df['a_id'].unique())

    num_users = len(user_list)
    num_anime = len(anime_list)

    user_id_to_index = {u_id: idx for idx, u_id in enumerate(user_list)}
    anime_id_to_index = {a_id: idx for idx, a_id in enumerate(anime_list)}

    R = cp.Variable((num_users, num_anime))

    delta = df[['u_id', 'a_id', 'score']]
    constraints = []

    for idx in range(len(delta)):
        u = user_id_to_index[int(delta.at[idx, 'u_id'])]
        i = anime_id_to_index[int(delta.at[idx, 'a_id'])]
        score = delta.at[idx, 'score']
        constraints.append(R[u, i] == score)

    obj = cp.Minimize(cp.normNuc(R))

    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.SCS)

    if prob.status == cp.OPTIMAL:
        print("Optimization succeeded.")
        return R.value
    else:
        print(f"Optimization failed with status: {prob.status}")
        return None

#This takes in matrix input
def nuclear_norm_model_matrix(data_matrix):
    num_users, num_anime = data_matrix.shape

    R = cp.Variable((num_users, num_anime))

    constraints = []
    for i in range(num_users):
        for j in range(num_anime):
            if data_matrix[i, j] != 0:
                constraints.append(R[i, j] == data_matrix[i, j])

    objective = cp.Minimize(cp.normNuc(R))
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS)

    if problem.status == cp.OPTIMAL:
        print("Optimization succeeded.")
        return R.value
    else:
        print(f"Optimization failed. Status: {problem.status}")
        return None


In [ ]:
completed_matrix = nuclear_norm_model_matrix(data_matrix)
